<a  href=https://towardsdatascience.com/how-i-would-explain-building-lightfm-hybrid-recommenders-to-a-5-year-old-b6ee18571309> Article 1 </a>

<a href=https://towardsdatascience.com/how-i-would-explain-building-lightfm-hybrid-recommenders-to-a-5-year-old-b6ee18571309> Article 2 </a>

<a href=https://towardsdatascience.com/using-cosine-similarity-to-build-a-movie-recommendation-system-ae7f20842599> Article 3 </a>

In [26]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
#Path to the folder containing data
data_path = "./julian_data"

In [5]:
recipes_raw = pd.read_csv(os.path.join(data_path, "RAW_recipes.csv"))

In [6]:
interactions_raw = pd.read_csv(os.path.join(data_path, "RAW_interactions.csv"))

In [7]:
interactions_raw.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [8]:
#Shorten dataframe to only include relevant columns
recipes = recipes_raw[['name','id','ingredients','n_ingredients']]
recipes.head()

,name,id,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,"['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [9]:
interactions = interactions_raw[['user_id','recipe_id','rating']]

In [13]:
interactions

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5
...,...,...,...
1132362,116593,72730,0
1132363,583662,386618,5
1132364,157126,78003,5
1132365,53932,78003,4


In [55]:
sample = recipes[:10]

In [15]:
test =recipes[:10]['ingredients'].apply(lambda x: eval(x))

In [42]:
test[9]

['sugar',
 'unsalted butter',
 'bananas',
 'eggs',
 'fresh lemon juice',
 'orange rind',
 'cake flour',
 'baking soda',
 'salt']

Test to see if mlb works

In [18]:
mlb = MultiLabelBinarizer()

In [19]:
mlb.fit(test)

MultiLabelBinarizer(classes=None, sparse_output=False)

In [24]:
recipe_test = [['sugar', 'unsalted butter', 'bananas', 'eggs','fresh lemon juice']]

In [20]:
ingredients_transformed = mlb.transform(test)

Get similarity between each recipe and recipe_test

In [28]:
recipe_test_trans

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [52]:
recipe_test_trans = mlb.transform(recipe_test)
sims = []
for recipe in ingredients_transformed:
    sim = cosine_similarity(recipe_test_trans,recipe.reshape(-1,78))
    sims.append(sim)

In [56]:
sims

[array([[0.]]),
 array([[0.18257419]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.15811388]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.18257419]]),
 array([[0.74535599]])]

In [58]:
sample['sims'] = sims

C:\Users\mahpx\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
sample['sims_unpacked'] = sample['sims'].apply(lambda x: x[0][0])

C:\Users\mahpx\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
#Sort recipes from greatest to least similarity
sample.sort_values('sims_unpacked',ascending=False)

,name,id,ingredients,n_ingredients,sims,sims_unpacked
9,beat this banana bread,75452,"['sugar', 'unsalted butter', 'bananas', 'eggs'...",9,[[0.7453559924999298]],0.745356
1,a bit different breakfast pizza,31490,"['prepared pizza crust', 'sausage patty', 'egg...",6,[[0.18257418583505539]],0.182574
8,bananas 4 ice cream pie,70971,"['chocolate sandwich style cookies', 'chocolat...",6,[[0.18257418583505539]],0.182574
4,amish tomato ketchup for canning,44061,"['tomato juice', 'apple cider vinegar', 'sugar...",8,[[0.15811388300841894]],0.158114
0,arriba baked winter squash mexican style,137739,"['winter squash', 'mexican seasoning', 'mixed ...",7,[[0.0]],0.000000
2,all in the kitchen chili,112140,"['ground beef', 'yellow onions', 'diced tomato...",13,[[0.0]],0.000000
3,alouette potatoes,59389,"['spreadable cheese with garlic and herbs', 'n...",11,[[0.0]],0.000000
5,apple a day milk shake,5289,"['milk', 'vanilla ice cream', 'frozen apple ju...",4,[[0.0]],0.000000
6,aww marinated olives,25274,"['fennel seeds', 'green olives', 'ripe olives'...",9,[[0.0]],0.000000
7,backyard style barbecued ribs,67888,"['pork spareribs', 'soy sauce', 'fresh garlic'...",22,[[0.0]],0.000000


In [ ]:
#The most similar is the one the one I based the recipe_test off of. It contains the first 5 ingredients.
#The second similar one however

In [69]:
sample.sort_values('sims_unpacked',ascending=False).loc[1,['name','ingredients']]['ingredients']

"['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']"

In [70]:
#So the second msot similar recipe isn't very similar, but it does make sense why there was overlap. Thats because it had eggs